In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os

import time

urlList = set()
areaList = set()

basicArea = ['jiangan', 'jianghan', 'qiaokou', 'dongxihu', 'wuchang', 'qingshan', 'hongshan', 'donghugaoxin',
             'jiangxia', 'caidian', 'huangbei', 'xinzhou','zhuankoukaifaqu']
headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWekit 537.36 (KHTML, like Gecko) Chrome",
            "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"}

def getSubUrl(basicArea):
    global areaList
    for area in basicArea:
        print('现在进行到' + area)
        url = 'http://wh.lianjia.com/ershoufang/' + area
        areaHtml = urlopen(url)
        bsObj = BeautifulSoup(areaHtml)
        print(url)
        aList = bsObj.select('div[data-role="ershoufang"]')[0].findAll('div')[1].findAll('a')
        for a in aList:
            if a['href'] not in areaList:
                areaList.add(a['href'][12:-1])
    with open('./lianjia/subAreaList', 'a+') as subAreaList:
        for area in areaList:
            subAreaList.write(area + '\n')

def scrapHouse(areaList):
    global urlList
    for area in areaList:
        with open('./lianjia/' + area, 'a+') as txtFile:
            basicUrl = 'https://wh.lianjia.com/ershoufang/' + area
            print('\n\n现在进行到' + area )
            pageNum = getPageNum(basicUrl)
            for num in range(1, pageNum+1):
                url = basicUrl + '/pg' + str(num)
                html = urlopen(url)
                try:
                    html = urlopen(url)
                    bsObj = BeautifulSoup(html)
                    listLen = len(bsObj.select('ul[class^="sell"]')[0].findAll('li'))
                    linkdivs = bsObj.select('div[class=title]')[1:listLen+1]
                    for div in linkdivs:
                        link = div.a['href']
                        if link not in urlList:
                            urlList.add(link)
                            txtFile.write(link + '\n')
                    print('地区:' + area + '第' + str(num) + '页爬取成功\n')
                except:
                    print('地区:' + area + '第' + str(num) + '页爬取失败\n')


def getPageNum(url):
    html = urlopen(url)
    bsObj = BeautifulSoup(html)
    num = eval(bsObj.select('div[page-data]')[0]['page-data'])['totalPage']
    return num

def updateData():
    pass

#getSubUrl(basicArea)
#scrapHouse(areaList)

In [3]:
import csv
import codecs
from bs4 import BeautifulSoup
import os
from urllib.request import urlopen
import re
headList = ['房屋类型', '所在楼层', '建筑面积', '户型结构', '套内面积', '建筑类型', '房屋朝向', '建筑结构', '装修情况', '梯户比例', '是否配备电梯', '产权年限', '挂牌时间', '交易权属', '上次交易', '房屋用途', '是否满两年', '产权所属(是否共有)', '抵押信息', '房本备件', '小区名', '首付', '税费', '挂牌价', '单价','url']
# sourceFile = './lianjia/baibuting'
def scrapData(sourceFile, targetCSVFile = './lianjia.csv'):
    with codecs.open(targetCSVFile, 'a+', 'utf-8') as csvFile:
        writer = csv.writer(csvFile)
        #writer.writerow(headList)
        with open(sourceFile , 'r') as urlFile:
            urlList = urlFile.read().split('\n')
        for url in urlList:
            print('正在进行' + url)
            try:
                html = urlopen(url)
                bsObj = BeautifulSoup(html)
                writer.writerow(cleanData(bsObj, url))
                #print('url:' + url + '成功')
            except:
                print('\nurl:' + url + '失败\n')
                
# 写csv文件的头
def writeHead(targetCSVFile='./lianjia.csv'):
    with codecs.open(targetCSVFile, 'w', 'utf-8') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(headList)
sourceFile = './lianjia/baibuting'
#scrapData(sourceFile, './experiment.csv')

In [4]:
# 数据整理
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
def cleanData(bsObj, url):
    data = list()
    # 基本信息
    a = bsObj.select_one('div[class="base"] div[class="content"]').get_text().replace(' ', '').replace('\n\n', '').split('\n')
    # 交易属性
    b = bsObj.select_one('div[class="transaction"] div[class="content"]').get_text().replace(' ', '').replace('\n', ' ').replace('  ', ' ').replace('  ', ' ').split(' ')[1:-1][1::2]   
    # 小区位置
    c = bsObj.select_one('div[class="communityName"] a').get_text()
    # 税费
    d = bsObj.select_one('span[class="taxtext"]').get_text()
    # 总价
    e = bsObj.select_one('span[class="total"]').get_text()
    # 单价
    f = bsObj.select_one('span[class="unitPriceValue"]').get_text()
    for num in range(0, len(a)):
        if num == 2 or num == 4 or num == 11:
            if re.findall('[0-9.]+', a[num]):
                data += re.findall('[0-9.]+', a[num])
            else:
                data.append(a[num][4:])
        #elif num == 10:
        #    if a[num] == '有':
        #        data.append(True)
        #    else:
        #        data.append(False)
        else:
            data.append(a[num][4:])
    #if b[4] == '未满两年':
    #    b[4] = False
    #else:
    #    b[4] = True
    #if b[5] == '非共有':
    #    b[5] = False
    #else:
    #    b[5] = True
    data += b
    data.append(c)
    data += re.findall('[0-9.]+',d)
    data += re.findall('[0-9.]+',e)
    data += re.findall('[0-9.]+',f)   
    #numlist = [2,4,11,21,22,23,24]
    #for num in numlist:
    #    data[num] = float(data[num])
    data.append(url)
    return data
#html= urlopen('https://wh.lianjia.com/ershoufang/104101085322.html')

#obj = BeautifulSoup(html)
#print(obj)
#print(cleanData(BeautifulSoup(html), 'https://wh.lianjia.com/ershoufang/104101085322.html'))
# 房屋类型, 所在楼层, 建筑面积, 户型结构, 套内面积, 建筑类型, 房屋朝向, 建筑结构, 装修情况, 梯户比例, 是否配备电梯, 产权年限, 挂牌时间, 交易权属, 上次交易, 房屋用途, 是否满两年, 产权所属(是否共有), 抵押信息, 房本备件, 小区名, 首付, 税费, 挂牌价, 单价      

In [5]:
import os
#for root, dirs, files in os.walk('.', topdown=False):
#    #print(root)
#    #print(dirs)
#    print(files)
#
os.listdir('./lianjia/')
#writeHead('./finalData.csv')
dir = './lianjia/'
for datasource in os.listdir(dir):
    path = dir + datasource
    print(path)
    if os.path.isfile(dir + datasource):
        scrapData(dir + datasource, targetCSVFile = './finalData.csv')

./lianjia/jiedaokou
正在进行https://wh.lianjia.com/ershoufang/104100984001.html
正在进行https://wh.lianjia.com/ershoufang/104100743927.html
正在进行https://wh.lianjia.com/ershoufang/104100994986.html
正在进行https://wh.lianjia.com/ershoufang/104100710866.html
正在进行https://wh.lianjia.com/ershoufang/104101010898.html
正在进行https://wh.lianjia.com/ershoufang/104101003678.html
正在进行https://wh.lianjia.com/ershoufang/104101102466.html
正在进行https://wh.lianjia.com/ershoufang/104100845879.html
正在进行https://wh.lianjia.com/ershoufang/104100897687.html
正在进行https://wh.lianjia.com/ershoufang/104101190344.html
正在进行https://wh.lianjia.com/ershoufang/104100942778.html
正在进行https://wh.lianjia.com/ershoufang/104101010949.html
正在进行https://wh.lianjia.com/ershoufang/104101021302.html
正在进行https://wh.lianjia.com/ershoufang/104100806730.html
正在进行https://wh.lianjia.com/ershoufang/104101156887.html
正在进行https://wh.lianjia.com/ershoufang/104101157614.html
正在进行https://wh.lianjia.com/ershoufang/104100930388.html
正在进行https://wh.lianjia.com/e